In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

In [28]:
# Load the Data.csv exported in WeatherPy to a DF
cities_df = pd.read_csv(r'..\python-api-challenge\output_data\data.csv')
cities_df.head()

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Isangel,-19.55,169.27,72.72,79,100,11.27,VU,1605982941
1,Port Alfred,-33.59,26.89,77.14,56,60,14.43,ZA,1605982942
2,Puerto Ayora,-0.74,-90.35,77.00,73,75,18.34,EC,1605982644
3,Faanui,-16.48,-151.75,80.01,72,57,9.48,PF,1605982611
4,Sur,22.57,59.53,71.74,54,0,3.31,OM,1605982572


In [7]:
#-------------Humidity Heatmap--------------

In [8]:
# Configure gmaps
gmaps.configure(api_key = g_key)

In [16]:
# Use the lat and lng as locations and Humidity as the weight
location = cities_df[['Lat', 'Long']]
humidity = cities_df['Humidity']
# location.head()
# humidity.head()

In [31]:
# Add Heatmap layer to map
# layout = {'width': '1000px',
#           'height': '500px',
#           'border': '1px solid black',
#           'padding': '1px',
#           'margin': '0 auto 0 auto'
#          }
fig = gmaps.figure()
heat_map = gmaps.heatmap_layer(location, 
                               weights = humidity,
                               dissipating = False,
                               max_intensity = 100,
                               point_radius = 2)
fig.add_layer(heat_map)
fig

Figure(layout=FigureLayout(height='420px'))

In [32]:
#Narrow down DF to find weather conditions of:
# - Max temp between 70 and 80
# - Wind Speed less than 10 mph
# - Zero Cloudiness
# - Drop rows that do not meet all conditions

In [41]:
ideal_temp = (cities_df["Max Temp"] > 70) & (cities_df["Max Temp"] < 80)
ideal_wind = cities_df["Wind Speed"] < 10
ideal_cloudiness = cities_df["Cloudiness"] == 0
ideal_set = ideal_temp & ideal_wind & ideal_cloudiness
ideal_df = cities_df[ideal_set].dropna()
# len(ideal_df)
ideal_df.head()

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,Sur,22.57,59.53,71.74,54,0,3.31,OM,1605982572
33,Tacoronte,28.48,-16.41,73.00,48,0,9.17,ES,1605982978
75,Los Llanos de Aridane,28.66,-17.92,75.20,44,0,4.70,ES,1605983024
85,Yura,-16.25,-71.68,71.60,14,0,9.17,PE,1605983036
106,Saint-Philippe,-21.36,55.77,75.20,60,0,3.36,RE,1605983062


In [39]:
# ---------------------Hotel Map--------------------

In [42]:
# Store ideal.df into new variable named hotel_df
hotel_df = ideal_df

# Add a "Hotel Name" column to the DF
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
4,Sur,22.57,59.53,71.74,54,0,3.31,OM,1605982572,
33,Tacoronte,28.48,-16.41,73.00,48,0,9.17,ES,1605982978,
75,Los Llanos de Aridane,28.66,-17.92,75.20,44,0,4.70,ES,1605983024,
85,Yura,-16.25,-71.68,71.60,14,0,9.17,PE,1605983036,
106,Saint-Philippe,-21.36,55.77,75.20,60,0,3.36,RE,1605983062,


In [59]:
# Set parameters to search for hotels within 5000 meters
params = {"radius" : 5000,
         "type" : "hotel",
         "keyword" : "hotel",
         "key" : g_key
         }

# set a base url for searching paramaters
search_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json" 


In [62]:
# Create loop to search for hotels in cities
# hotels = []

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    long = row["Long"]
    lat_long = f"{lat},{long}"
    params["location"] = lat_long
    
    response = requests.get(search_url, params = params).json()
    results = response["results"]
    
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
    except:
        print("hotel not found")

hotel_df

hotel not found
hotel not found
hotel not found
hotel not found


,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
4,Sur,22.57,59.53,71.74,54,0,3.31,OM,1605982572,Sur Plaza Hotel
33,Tacoronte,28.48,-16.41,73.00,48,0,9.17,ES,1605982978,OYO San Diego
75,Los Llanos de Aridane,28.66,-17.92,75.20,44,0,4.70,ES,1605983024,Hotel Hacienda de Abajo
85,Yura,-16.25,-71.68,71.60,14,0,9.17,PE,1605983036,Hotel Not Found
106,Saint-Philippe,-21.36,55.77,75.20,60,0,3.36,RE,1605983062,Les Embruns Du Baril
145,Abu Dhabi,24.47,54.37,78.80,61,0,4.70,AE,1605983108,Villaggio Hotel & Resort
151,Kidal,18.44,1.41,73.98,19,0,8.19,ML,1605982797,Hotel Not Found
172,Puerto del Rosario,28.50,-13.86,73.40,53,0,8.05,ES,1605983137,Hotel El Mirador
224,Keti Bandar,24.14,67.45,73.17,42,0,6.69,PK,1605983194,Hotel Not Found
238,Rivadavia,-33.19,-68.46,78.80,26,0,6.93,AR,1605983209,Hotel Pasambay
